In [ ]:
from glob import glob
import fnmatch
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
imagePatches = glob('IDC_regular_ps50_idx5/**/*.png', recursive=True)
patternZero = '*class0.png'
patternOne = '*class1.png'

In [ ]:
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)

In [ ]:
def process_images(lowerIndex,upperIndex):

    height = 50
    width = 50
    channels = 3
    x = [] 
    y = [] 
    for img in imagePatches[lowerIndex:upperIndex]:
        
        full_size_image = cv2.imread(img)
        
        image = (cv2.resize(full_size_image, (width,height), interpolation=cv2.INTER_CUBIC))
       
        x.append(image)
        if img in classZero:
            y.append(0)
        elif img in classOne:
            y.append(1)
        else:
            return
    return x,y

In [ ]:
X,Y = process_images(0,100000)

In [ ]:
X = np.array(X)
X = X.astype(np.float32) 
X /= 255.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.15)

In [ ]:
Y.count(0)

70901

In [ ]:
y_train.count(1)  

24844

In [ ]:
y_train.count(0)

60156

In [ ]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Using TensorFlow backend.


In [ ]:
X_trainShape = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]
X_testShape = X_test.shape[1]*X_test.shape[2]*X_test.shape[3]
X_trainFlat = X_train.reshape(X_train.shape[0], X_trainShape)
X_testFlat = X_test.reshape(X_test.shape[0], X_testShape)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler 
random_under_sampler = RandomUnderSampler(ratio='majority')
X_trainRos, Y_trainRos = random_under_sampler.fit_sample(X_trainFlat, y_train)
X_testRos, Y_testRos = random_under_sampler.fit_sample(X_testFlat, y_test)

In [ ]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout
batch_size = 256
num_classes = 2
epochs = 80
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                 activation='relu',
                 input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Flatten()) #this converts our 3D feature maps to 1D feature vectors for the dense layer below
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))

In [ ]:
import keras
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.00001),
              metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=180,
    horizontal_flip=True,vertical_flip = True)

In [ ]:
from keras.callbacks import ModelCheckpoint,  EarlyStopping
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=3, mode='min')

In [ ]:
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
Y_trainRosHot = to_categorical(Y_trainRos, num_classes = 2)
Y_testRosHot = to_categorical(Y_testRos, num_classes = 2)

In [ ]:
for i in range(len(X_trainRos)):
    height, width, channels = 50,50,3,
    X_trainRosReshaped = X_trainRos.reshape(len(X_trainRos),height,width,channels)

In [ ]:
for i in range(len(X_testRos)):
    height, width, channels = 50,50,3
    X_testRosReshaped = X_testRos.reshape(len(X_testRos),height,width,channels)

In [ ]:
training = model.fit_generator(datagen.flow(X_trainRosReshaped,Y_trainRosHot,batch_size=batch_size),steps_per_epoch=len(X_trainRosReshaped) / batch_size, epochs=epochs,validation_data=(X_testRosReshaped, Y_testRosHot), verbose=1, callbacks=[early_stopping_monitor, model_checkpoint])

Epoch 1/80


C:\Users\Hanane\AppData\Local\Continuum\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Hanane\AppData\Local\Continuum\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


195/194 [==============================] - 128s 657ms/step - loss: 0.6852 - accuracy: 0.5405 - val_loss: 0.6547 - val_accuracy: 0.6572

Epoch 00001: val_loss improved from inf to 0.65475, saving model to best_model.h5
Epoch 2/80
195/194 [==============================] - 126s 647ms/step - loss: 0.6066 - accuracy: 0.6922 - val_loss: 0.5594 - val_accuracy: 0.7212

Epoch 00002: val_loss improved from 0.65475 to 0.55943, saving model to best_model.h5
Epoch 3/80
195/194 [==============================] - 132s 675ms/step - loss: 0.5364 - accuracy: 0.7497 - val_loss: 0.5230 - val_accuracy: 0.7561

Epoch 00003: val_loss improved from 0.55943 to 0.52301, saving model to best_model.h5
Epoch 4/80
195/194 [==============================] - 137s 700ms/step - loss: 0.5263 - accuracy: 0.7576 - val_loss: 0.5186 - val_accuracy: 0.7599

Epoch 00004: val_loss improved from 0.52301 to 0.51862, saving model to best_model.h5
Epoch 5/80
195/194 [==============================] - 139s 715ms/step - loss: 0.520

In [ ]:
from keras.models import load_model
from sklearn import metrics
model = load_model('best_model.h5')
y_pred_one_hot = model.predict(X_testRosReshaped)
y_pred_labels = np.argmax(y_pred_one_hot, axis = 1)
y_true_labels = np.argmax(Y_testRosHot,axis=1)

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_true=y_true_labels, y_pred=y_pred_labels)
print(confusion_matrix)

[[3191 1064]
 [ 815 3440]]
